In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
df = pd.read_csv('./dataset/data_file.csv', sep=',')
df

FileName  \
0               0124e21d-018c-4ce0-92a3-b9e205a76bc0.dll   
1           05c8318f98a5d301d80000009c316005.vertdll.dll   
2               06054fba-5619-4a86-a861-ffb0464bef5d.dll   
3            075822ac99a5d301660400009c316005.adhapi.dll   
4      090607dd9ba5d301ca0900009c316005.SensorsNative...   
...                                                  ...   
62480        VirusShare_a43ceb5e5fffc793e0205d15a0606cb0   
62481        VirusShare_0190dafc8304111a00fccf57340ea6a4   
62482        VirusShare_0f3ca55979aaf59158d6b01140696e44   
62483        VirusShare_fca5ce35f1690db6babca5aa5d559535   
62484        VirusShare_d7955a7e6d1e16800feafd4204cbac2b   

                                md5Hash  Machine  DebugSize  DebugRVA  \
0      79755c51e413ed3c6be4635fd729a6e1      332          0         0   
1      95e19f3657d34a432eada93221b0ea16    34404         84    121728   
2      85c32641d77a54e19ba8ea4ab305c791      332          0         0   
3      62e3b959d982ef534b66f819fe15f085    34404         84     19904   
4      ae38c5f7d313ad0ff3bfb8826476767f    34404         84     97728   
...                                 ...      ...        ...       ...   
62480  a43ceb5e5fffc793e0205d15a0606cb0      332          0         0   
62481  0190dafc8304111a00fccf57340ea6a4      332          0         0   
62482  0f3ca55979aaf59158d6b01140696e44      332          0         0   
62483  fca5ce35f1690db6babca5aa5d559535      332          0         0   
62484  d7955a7e6d1e16800feafd4204cbac2b      332          0         0   

       MajorImageVersion  MajorOSVersion  ExportRVA  ExportSize  IatVRA  \
0                      0               4          0           0    8192   
1                     10              10     126576        4930       0   
2                      0               4          0           0    8192   
3                     10              10      21312         252   18160   
4                     10              10     105792        1852   70592   
...                  ...             ...        ...         ...     ...   
62480                  1               4          0           0    4096   
62481                  7              10          0           0       0   
62482                  0               4          0           0  404908   
62483                  0               4      14448          70    4096   
62484                  0               5          0           0    4096   

       MajorLinkerVersion  MinorLinkerVersion  NumberOfSections  \
0                       8                   0                 3   
1                      14                  10                 8   
2                       8                   0                 3   
3                      14                  10                 6   
4                      14                  10                 7   
...                   ...                 ...               ...   
62480                   6                   0                 3   
62481                   7                   0                 7   
62482                   2                  50                11   
62483                   8                   0                 4   
62484                   9                   0                 6   

       SizeOfStackReserve  DllCharacteristics  ResourceSize  BitcoinAddresses  \
0                 1048576               34112           672                 0   
1                  262144               16864          1024                 0   
2                 1048576               34112           672                 0   
3                  262144               16736          1040                 0   
4                  262144               16736          1096                 0   
...                   ...                 ...           ...               ...   
62480             1048576                   0         23504                 0   
62481             1048576                   0         15704                 0   
62482          

In [18]:
header_names = ['FileName', 'md5Hash', 'Machine', 'DebugSize', 'DebugRVA', 'MajorImageVersion', 'MajorOSVersion', 
                'ExportRVA', 'ExportSize', 'IatVRA', 'MajorLinkerVersion', 'MinorLinkerVersion', 'NumberOfSections', 
                'SizeOfStackReserve', 'DllCharacteristics', 'ResourceSize', 'BitcoinAddresses', 'Benign']

categorical_columns = ['Machine', 'MajorImageVersion', 'MajorOSVersion', 'MajorLinkerVersion', 'MinorLinkerVersion', 'DllCharacteristics', 'NumberOfSections']
df[categorical_columns] = df[categorical_columns].astype('category')

binary_columns = ['Benign', 'BitcoinAddresses']

numeric_columns = df.select_dtypes(include=['int64']).columns.difference(binary_columns)

## Step 3: Data Preprocessing and Data Processing

Based on our data exploration, we observed that the two variables, 'FileName' and 'md5Hash', have unique values for each object, and therefore, we remove them from the dataset beforhand.

In [19]:
df.drop(columns=['FileName', 'md5Hash'], inplace=True)

Based on our understanding of the meaning of 'MinorLinkerVersion' and 'MajorLinkerVersion', we combine them into a single feature called 'LinkerVersion'.

In [20]:
df['LinkerVersion'] = df['MajorLinkerVersion'].astype(str) + '.' + df['MinorLinkerVersion'].astype(str)
df['LinkerVersion'] = df['LinkerVersion'].astype('category')
df.drop(columns=['MajorLinkerVersion', 'MinorLinkerVersion'], inplace=True)

categorical_columns = [col for col in categorical_columns if col not in ['MajorLinkerVersion', 'MinorLinkerVersion']]
categorical_columns.append('LinkerVersion')

In [21]:
df.head()

Machine  DebugSize  DebugRVA MajorImageVersion MajorOSVersion  ExportRVA  \
0     332          0         0                 0              4          0   
1   34404         84    121728                10             10     126576   
2     332          0         0                 0              4          0   
3   34404         84     19904                10             10      21312   
4   34404         84     97728                10             10     105792   

   ExportSize  IatVRA NumberOfSections  SizeOfStackReserve DllCharacteristics  \
0           0    8192                3             1048576              34112   
1        4930       0                8              262144              16864   
2           0    8192                3             1048576              34112   
3         252   18160                6              262144              16736   
4        1852   70592                7              262144              16736   

   ResourceSize  BitcoinAddresses  Benign LinkerVersion  
0           672                 0       1           8.0  
1          1024                 0       1         14.10  
2           672                 0       1           8.0  
3          1040                 0       1         14.10  
4          1096                 0       1         14.10

### Test 1: Consider only numerical variables

### Test 2: Consider only categorical variables